# Linear Regression

1. Prepare Data  
  
2. Designing the model
   <ol type="i">
      <li>input size</li>
      <li>output size</li>
      <li>forward pass</li>
   </ol>  
   
3. Construct loss and optimizer  

4. Training Loop
   <ol type="i">
      <li>forward pass: compute prediction and loss</li>
      <li>backward pass: gradients</li>
      <li>update weights</li>
   </ol>

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### 0. Prepare Data

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train_unscaled)
X_test = sc.transform(X_test_unscaled)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

### 1. Desining the Model

In [3]:
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

### 2. Loss Function and the Optimizer

In [4]:
learning_rate = 0.01

Error = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 3. Training Loop

In [5]:
num_epochs = 100

# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = Error(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.7033
epoch: 20, loss = 0.5440
epoch: 30, loss = 0.4515
epoch: 40, loss = 0.3921
epoch: 50, loss = 0.3507
epoch: 60, loss = 0.3200
epoch: 70, loss = 0.2962
epoch: 80, loss = 0.2771
epoch: 90, loss = 0.2614
epoch: 100, loss = 0.2483


### 4. Evaluations

In [6]:

#Evaluations should be done without the gradients being tracked.
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.8860


## Foot Notes